In [10]:
# Dependencies
import requests
import time
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
# not condoned
import warnings
warnings.filterwarnings("ignore")

In [11]:
# splinter set-up
# !which chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

In [12]:
# helper functions

# browser status code not implemented, so attempting wait on content(css selector) of interest available
# or timeout after 10 sec
def waitforload(url, selector):
    browser.visit(url)
    t1 = time.perf_counter()
    while browser.is_element_not_present_by_css(selector):
        t2 = time.perf_counter()
        if (t2-t1)>10:
            return ""
        # wait?
    return browser.html

def requestsoup(url):
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text,'html.parser')
    return soup

---
### NASA Mars News
[https://mars.nasa.gov/news/](https://mars.nasa.gov/news/)

In [13]:
# NASA Mars News
news_url = 'https://mars.nasa.gov/news/'

html = waitforload(news_url, 'div[class="content_title"]')

if html!="":
    # news_title = browser.find_by_css('div[class="content_title"] a')[0].text   
    # news_p = browser.find_by_css('div[class="article_teaser_body"]')[0].text   
    # soup parse faster
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find('div', 'list_text')
    news_title = results.find('div','content_title').find('a').text
    news_p = results.find('div','article_teaser_body').text
else:
    news_title = "Latest News Currently Unavailable"
    news_p = "The connection timed out. Please check back later."

print(news_title)
print(news_p)

InSight Captures Sunrise and Sunset on Mars
InSight joins the rest of NASA's Red Planet surface missions, all of which have photographed either the start or end of a Martian day.


---
### JPL Mars Space Images - Featured Image
[https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars)

In [28]:
# JPL Mars Space Images - Featured Image
qurl = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
url = qurl.split('/spaceimages')[0]

soup = requestsoup(qurl)

try:
    iurl = url + soup.find('a', id='full_image')["data-link"]
    soup = requestsoup(iurl)
    ilink = soup.find('img', 'main_image')
    if ilink!=None:
        featured_image_url = url+ilink["src"]
    else:
        featured_image_url = ""
        print('not finding featured image src')
except:
    featured_image_url = ""
    print('not finding a tag with data-link attr')

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17550_hires.jpg'

---
### Mars Weather
[https://twitter.com/marswxreport?lang=en](https://twitter.com/marswxreport?lang=en)

In [29]:
# Mars Weather
url = 'https://twitter.com/marswxreport?lang=en'
    
soup = requestsoup(url)
tweet = soup.find('p','tweet-text')

try:
    tweet = tweet.text.split('pic.twitter')[0]
    mars_weather = ' '.join(tweet.split('\n'))
except Exception as e:
    mars_weather = ""
    print('tweet not found or has no text')
    
mars_weather

'InSight sol 155 (2019-05-04) low -99.3ºC (-146.8ºF) high -18.8ºC (-1.8ºF) winds from the SW at 4.5 m/s (10.1 mph) gusting to 14.3 m/s (31.9 mph) pressure at 7.40 hPa'

---
### Mars Facts
[https://space-facts.com/mars/](https://space-facts.com/mars/)

In [34]:
# Mars Facts
url = 'https://space-facts.com/mars/'

tables = pd.read_html(url)
try:
    df = tables[0]
    df.set_index(0, inplace=True)
    del df.index.name
    html_table = df.to_html(header=False)
    html_table=html_table.replace('\n', '')
except Exception as e:
    html_table=""
    print('not finding tables')
    
html_table

'<table border="1" class="dataframe">  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

---
### Mars Hemispheres
[https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars)

In [33]:
# Mars Hemispheres
surl = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
url = surl.split('/search')[0]

html = waitforload(surl, 'div[class="description"]')
# this page sometimes takes a long time to load, or fails to load

if (html!=""):
    soup = BeautifulSoup(html, 'html.parser')
    divs = soup.find_all('div','description')
    hrefs = [d.find('a')['href'] for d in divs]

    hemisphere_image_urls = []
    for href in hrefs:
        furl = url+href
        soup = requestsoup(furl)
        try:
            title = soup.find('h2','title').text
            img_url = soup.find('div','downloads').find('a')['href']
            hemisphere_image_urls.append({"title":title, "img_url":img_url})
        except Exception as e:
            print(f'error finding title or image at {furl}')
            continue

    
for rec in hemisphere_image_urls:
    print(rec)
    

{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
{'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
{'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}
{'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}
